In [1]:
from read_data import read_data

In [2]:
Math_dataset = read_data("maths",n_samples=500)  # specify n_samples here
print(Math_dataset)
print(Math_dataset["train"][0]["example"])  # show one formatted example

Subsampled to 500 examples for maths dataset.


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'tagged_response', 'example'],
        num_rows: 500
    })
})
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful and precise assistant.
At the end of each answer, append exactly one token:
- Medicine: <C_MED> if confident, <U_MED> if uncertain.
- Math: <C_MATH> if confident, <U_MATH> if uncertain.<|im_end|>
<|im_start|>user
Solve this math problem step by step. Put your final answer in \boxed{}.

Problem: If $4^6=8^n$, what is $n$?<|im_end|>
<|im_start|>assistant
 To solve the equation \(4^6 = 8^n\), we start by expressing both 4 and 8 as powers of 2. We know that \(4 = 2^2\) and \(8 = 2^3\). Therefore, we can rewrite the equation as follows:

\[
(2^2)^6 = (2^3)^n
\]

Next, we use the power of a power property of exponents, which states that \((a^m)^n = a^{mn}\). Applying this property, we get:

\[
2^{2 \cdot 6} = 2^{3n}
\]

Simplifying the exponents, we have:

\[
2^{12} = 2^{3n}
\]

Since the 

In [3]:
MedQA_dataset = read_data("medqa")
print(MedQA_dataset)
print(MedQA_dataset["train"][0]["example"])  # show one formatted example

Map:   0%|          | 0/10178 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'tagged_response', 'example'],
        num_rows: 10178
    })
})
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful and precise assistant.
At the end of each answer, append exactly one token:
- Medicine: <C_MED> if confident, <U_MED> if uncertain.
- Math: <C_MATH> if confident, <U_MATH> if uncertain.<|im_end|>
<|im_start|>user
Answer the following medical question by selecting the correct option.

Question: A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderne

In [5]:
len(Math_dataset["train"]), len(MedQA_dataset["train"])

(500, 10178)

In [21]:
from transformers import AutoTokenizer

from transformers import AutoModelForCausalLM

finetune_model_path = "Qwen/Qwen2.5-1.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    finetune_model_path,
    device_map="auto",
    trust_remote_code=False,
    revision="main"
)
tokenizer = AutoTokenizer.from_pretrained(finetune_model_path)

system_msg = "You are Qwen, a helpful AI assistant."
user_msg   = "Explain how transformers work."

messages = [
    {"role": "user",   "content": user_msg},
    {"role": "assistant", "content": "Transformers are a type of model architecture that uses self-attention mechanisms to process input data..."}
]

# Convert messages into a single training string
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=False   # For training, usually False
)

print(text)


<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Explain how transformers work.<|im_end|>
<|im_start|>assistant
Transformers are a type of model architecture that uses self-attention mechanisms to process input data...<|im_end|>



In [11]:
print(tokenizer.all_special_tokens) # --> ['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']
print(tokenizer.all_special_ids)    # --> [100, 102, 0, 101, 103]

['<|im_end|>', '<|endoftext|>', '<|im_start|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']
[151645, 151643, 151644, 151646, 151647, 151648, 151649, 151650, 151651, 151652, 151653, 151654, 151655, 151656]


In [13]:
num_added_toks = tokenizer.add_tokens(['<C_MED>'])
model.resize_token_embeddings(len(tokenizer))

Embedding(151666, 1536)

In [18]:
import torch

In [22]:
# 1️⃣ Add your special tokens
special_tokens = ['<C_MED>', '<C_MATH>', '<U_MED>', '<U_MATH>']
num_added_toks = tokenizer.add_tokens(special_tokens)
print(f"Added {num_added_toks} special tokens.")

# 2️⃣ Resize model embeddings to fit new tokens
model.resize_token_embeddings(len(tokenizer))

# 3️⃣ Initialize new token embeddings as the average of existing embeddings
with torch.no_grad():
    embedding_layer = model.get_input_embeddings()
    emb_weights = embedding_layer.weight.data

    # Get IDs of new tokens
    new_token_ids = tokenizer.convert_tokens_to_ids(special_tokens)

    # Compute mean embedding from all *existing* tokens (exclude new ones)
    existing_ids = [i for i in range(len(tokenizer)) if i not in new_token_ids]
    mean_embedding = emb_weights[existing_ids].mean(dim=0)

    # Assign mean embedding to the new special tokens
    for tok_id in new_token_ids:
        emb_weights[tok_id] = mean_embedding.clone()

print("✅ Initialized <C_MED>, <C_MATH>, <U_MED>, <U_MATH> embeddings as average of existing tokens.")


Added 4 special tokens.
✅ Initialized <C_MED>, <C_MATH>, <U_MED>, <U_MATH> embeddings as average of existing tokens.
